In [29]:
import numpy as np
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [24]:
n = 10
N = 2*n
t=1
mu=1
U=1

In [25]:
op = FermionicOp(
    {
        "+_0 -_0": 1.0,
        "+_1 -_1": -1.0,
    },
    num_spin_orbitals=2,
)

In [26]:
print(op)

Fermionic Operator
number spin orbitals=2, number terms=2
  1.0 * ( +_0 -_0 )
+ -1.0 * ( +_1 -_1 )


In [27]:
def qOp(i,j,N=20):
    return FermionicOp(
    {
        "+_{i} -_{j}".format(i=i%N,j=j%N): 1.0,
    },
    num_spin_orbitals=N,
)

In [28]:
t_term = 0
mu_term =0
U_term =0
for i in range(n):
    t_term+= qOp(2*i,2*i+2) + qOp(2*i+2,2*i) + qOp(2*i+1,2*i+3) + qOp(2*i+3,2*i+1) 
    mu_term += qOp(2*i,2*i) + qOp(2*i+1,2*i+1)
    U_term += (qOp(2*i,2*i) @ qOp(2*i+1, 2*i+i))
H = U * U_term - mu * mu_term - t * t_term

print(H)

Fermionic Operator
number spin orbitals=20, number terms=70
  1.0 * ( +_0 -_0 +_1 -_0 )
+ 1.0 * ( +_2 -_2 +_3 -_3 )
+ 1.0 * ( +_4 -_4 +_5 -_6 )
+ 1.0 * ( +_6 -_6 +_7 -_9 )
+ 1.0 * ( +_8 -_8 +_9 -_12 )
+ 1.0 * ( +_10 -_10 +_11 -_15 )
+ 1.0 * ( +_12 -_12 +_13 -_18 )
+ 1.0 * ( +_14 -_14 +_15 -_1 )
+ 1.0 * ( +_16 -_16 +_17 -_4 )
+ 1.0 * ( +_18 -_18 +_19 -_7 )
+ -1.0 * ( +_12 -_12 )
+ -1.0 * ( +_9 -_9 )
+ -1.0 * ( +_5 -_5 )
+ -1.0 * ( +_10 -_10 )
+ -1.0 * ( +_15 -_15 )
+ -1.0 * ( +_4 -_4 )
+ -1.0 * ( +_0 -_0 )
+ -1.0 * ( +_7 -_7 )
+ -1.0 * ( +_3 -_3 )
+ -1.0 * ( +_18 -_18 )
+ -1.0 * ( +_8 -_8 )
+ -1.0 * ( +_13 -_13 )
+ -1.0 * ( +_17 -_17 )
+ -1.0 * ( +_14 -_14 )
+ -1.0 * ( +_2 -_2 )
+ -1.0 * ( +_1 -_1 )
+ -1.0 * ( +_6 -_6 )
+ -1.0 * ( +_16 -_16 )
+ -1.0 * ( +_11 -_11 )
+ -1.0 * ( +_19 -_19 )
+ -1.0 * ( +_19 -_1 )
+ -1.0 * ( +_13 -_11 )
+ -1.0 * ( +_12 -_14 )
+ -1.0 * ( +_18 -_0 )
+ -1.0 * ( +_11 -_13 )
+ -1.0 * ( +_7 -_9 )
+ -1.0 * ( +_17 -_15 )
+ -1.0 * ( +_10 -_12 )
+ -1.0 * ( +_2 -_4 )
+

In [30]:
mapper = JordanWignerMapper()
qubit_jw_op = mapper.map(H)
print(qubit_jw_op)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIXYIIIII', 'IIIIIIIIIIIIIYYIIIII', 'IIIIIIIIIIIIIXXIIIII', 'IIIIIIIIIIIIIYXIIIII', 'IIIIIIIIIIIIIXYZIIII', 'IIIIIIIIIIIIIYYZIIII', 'IIIIIIIIIIIIIXXZIIII', 'IIIIIIIIIIIIIYXZIIII', 'IIIIIIIIIIXZYIIIIIII', 'IIIIIIIIIIYZYIIIIIII', 'IIIIIIIIIIXZXIIIIIII', 'IIIIIIIIIIYZXIIIIIII', 'IIIIIIIIIIXZYZIIIIII', 'IIIIIIIIIIYZYZIIIIII', 'IIIIIIIIIIXZXZIIIIII', 'IIIIIIIIIIYZXZIIIIII', 'IIIIIIIXZZYIIIIIIIII', 'IIIIIIIYZZYIIIIIIIII', 'IIIIIIIXZZXIIIIIIIII', 'IIIIIIIYZZXIIIIIIIII', 'IIIIIIIXZZYZIIIIIIII', 'IIIIIIIYZZYZIIIIIIII', 'IIIIIIIXZZXZIIIIIIII', 'IIIIIIIYZZXZIIIIIIII', 'IIIIXZZZYIIIIIIIIIII', 'IIIIYZZZYIIIIIIIIIII', 'IIIIXZZZXIIIIIIIIIII', 'IIIIYZZZXIIIIIIIIIII', 'IIIIXZZZYZIIIIIIIIII', 'IIIIYZZZYZIIIIIIIIII', 'IIIIXZZZXZIIIIIIIIII', 'IIIIYZZZXZIIIIIIIIII', 'IXZZZZYIIIIIIIIIIIII', 'IYZZZZYIIIIIIIIIIIII', 'IXZZZZXIIIIIIIIIIIII', 'IYZZZZXIIIIIIIIIIIII', 'IXZZZZYZIIIIIIIIIIII', '

In [31]:
from qiskit.algorithms import MinimumEigensolver, VQEResult
from qiskit.opflow import (
    CircuitSampler,
    CircuitStateFn,
    ExpectationBase,
    ExpectationFactory,
    ListOp,
    OperatorBase,
    PauliSumOp,
    StateFn,
)
from typing import Union, List, Dict

# Define a custome VQE class to orchestra the ansatz, classical optimizers, 
# initial point, callback, and final result
class CustomVQE(MinimumEigensolver):
    
    def __init__(self, estimator, circuit, optimizer, callback=None, initial_point = None, sym = True):
        self._estimator = estimator
        self.ansatz = circuit
        self._optimizer = optimizer
        self._callback = callback
        self._initial_point = initial_point
        backend = Aer.get_backend("statevector_simulator")
        self._circuit_sampler = CircuitSampler(
            backend
        )
        self._sym = sym
     
    def _get_eigenstate(self, optimal_parameters) -> Union[List[float], Dict[str, int]]:
        """Get the simulation outcome of the ansatz, provided with parameters."""
        optimal_circuit = self.ansatz.bind_parameters(optimal_parameters)
        state_fn = self._circuit_sampler.convert(StateFn(optimal_circuit)).eval()
        state = state_fn.primitive.data  # VectorStateFn -> Statevector -> np.array
        return state
    
    
    def compute_minimum_eigenvalue(self, operators, aux_operators=None):    
        num = 0
        # Define objective function to classically minimize over
        def objective(x, shots = None):
            #adjust number of shots
            # Execute job with estimator primitive
            if shots is not None:
                shots = int(shots)
                self._estimator.set_options(shots = shots)
            else:
                self._estimator.set_options(shots = 1024)
            job = self._estimator.run([self.ansatz], [operators], [x])
            # Get results from jobs
            est_result = job.result()
            # Get the measured energy value
            value = est_result.values[0]
            if self._sym:
                spinop = TotalSpin(self.ansatz.num_qubits)
                job_sym = self._estimator.run([self.ansatz], [spinop], [x])
                est_result_sym = job_sym.result()
                value_sym = est_result_sym.values[0]
                value += 0.5*(value_sym)**2
            # Save result information using callback function
            if self._callback is not None:
                self._callback(value)
            return value
            
        # Select an initial point for the ansatzs' parameters
        if self._initial_point is None:  
            x0 = np.pi/4 * np.random.rand(self.ansatz.num_parameters)
        else:
            x0 = self._initial_point
        # Run optimization
        res = self._optimizer.minimize(objective, x0=x0)
        
        # Populate VQE result
        result = VQEResult()
        result.cost_function_evals = res.nit
        result.eigenvalue = res.fun[0]
        result.optimal_parameters = res.x
        result.eigenstate = self._get_eigenstate(result.optimal_parameters)
        
        return result

/var/folders/n_/1wl7_2l52y3g1_82hvk9hw3h0000gn/T/ipykernel_30727/2981371494.py:1: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import MinimumEigensolver, VQEResult


In [33]:
from qiskit.circuit.library import EfficientSU2

    
ansatz = EfficientSU2(N, reps=3, skip_final_rotation_layer=True)

In [34]:
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit import Aer
from math import ceil,floor
# Define a simple callback function
def callback(value):
        intermediate_info.append(value)

#optimizer = SPSA(maxiter = 400)
optimizer = COBYLA(maxiter = 2000)
estimator = Estimator([ansatz], [H])
custom_vqe = CustomVQE(estimator, ansatz, optimizer, callback=callback, sym = False)

TypeError: Estimator.__init__() takes 1 positional argument but 3 were given

In [ ]:
from time import time
intermediate_info = []
start = time()
result = custom_vqe.compute_minimum_eigenvalue(ham)
end = time()

print(result)
print(f'execution time (s): {end - start:.2f}')

In [ ]:
plt.plot(intermediate_info, color='purple', lw=2, label='Simulated VQE')
plt.ylabel('Energy')
plt.xlabel('Iterations')
# Exact ground state energy value
#plt.axhline(y=gs_energy, color="tab:red", ls="--", lw=2, label="Target: " + str(gs_energy))
plt.legend()
plt.grid()
plt.show()